In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
path = "./input/"

df = pd.read_csv(path + "train.csv")
df_test = pd.read_csv(path + "test.csv")

data = pd.concat([df, df_test], sort=False)

print(df.shape)
print(df_test.shape)

(7134, 29)
(1794, 28)


In [4]:
# Create base line model
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error as MAE

str_list =["Id", "groupId", "matchId", "matchType"]

X, y = df.drop(str_list, axis=1).iloc[:, :-1], df.iloc[:, -1]
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.3, random_state=42)

In [6]:
# XGBoost baseline
import xgboost as xgb

xgr = xgb.XGBRegressor()
xgr.fit(X_train, y_train)
preds_xgr = xgr.predict(X_valid)

score_xgb = MAE(y_valid, preds_xgr)
print("Score with xgboost regressor:", score_xgb)

[22:00:53] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Score with xgboost regressor: 0.0679120909551406


In [7]:
train_DMatrix = xgb.DMatrix(data=X_train, label=y_train)
valid_DMatrix = xgb.DMatrix(data=X_valid, label=y_valid)

params = {"objective":"reg:linear", "max_depth":4}

xgr_cv = xgb.XGBRegressor()

cv_results = xgb.cv(dtrain=train_DMatrix, params=params, nfold=5, num_boost_round=10, metrics="mae", as_pandas=True, seed=42)

# preds = xgb.predict(X_valid)

print(cv_results)
print("*****************************")
print((cv_results["test-mae-mean"]))

[22:00:55] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[22:00:55] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[22:00:55] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[22:00:55] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[22:00:55] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
   train-mae-mean  train-mae-std  test-mae-mean  test-mae-std
0        0.192878       0.001234       0.193364      0.005698
1        0.150147       0.001086       0.151258      0.005023
2        0.121571       0.000741       0.123136      0.004249
3        0.102160       0.000760       0.104454      0.003944
4        0.089528       0.000848       0.092363      0.003325
5        0.081437       0.000615       0.084732 

In [8]:
params = {"objective":"reg:linear", "max_depth":5}

xg_reg = xgb.train(params=params, dtrain=train_DMatrix, num_boost_round=10)

[22:03:01] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [10]:
xgb.plot_tree(num_trees=0, booster=xg_reg)
plt.show()

ExecutableNotFound: failed to execute ['dot', '-Tpng'], make sure the Graphviz executables are on your systems' PATH